In [208]:
## import packages
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## hide warnings
import warnings
warnings.filterwarnings('ignore')

In [402]:
## load the telangana fires data into data frame
## proprecess the data to keep only year and geometry
csv_PATH = "/Users/yaodongqiu/Desktop/Data4Policy/Geospatial Data Science Internship/Datasets/telangana_fires.csv"
telangana_fires_orginal_df = pd.read_csv(csv_PATH)
telangana_fires = telangana_fires_orginal_df[["latitude","longitude","acq_date"]]
telangana_fires["geometry"] = gpd.points_from_xy(telangana_fires["longitude"], telangana_fires["latitude"])
telangana_fires["year"] = "20"+telangana_fires.acq_date.str[6:8]
telangana_fires = gpd.GeoDataFrame(telangana_fires[["year","geometry"]])
telangana_fires.head()

,year,geometry
0,2015,POINT (80.53140 18.37770)
1,2015,POINT (80.98930 17.58980)
2,2015,POINT (81.16120 17.60280)
3,2015,POINT (80.31350 17.52410)
4,2015,POINT (80.34040 17.75710)


In [399]:
## load district labeled data and select data with "NAME_1" = Telangana
## transform the data with only district name and the geometry columns

PATH_shape = "/Users/yaodongqiu/Desktop/gadm40_IND_shp/gadm40_IND_3.shp"
india_shapefile = gpd.read_file(PATH_shape)
Telangana_orginal = india_shapefile[india_shapefile["NAME_1"] == "Telangana"]
Telangana = Telangana_orginal[["NAME_3","geometry"]]

In [400]:
## join two and label the fire data with location if the fire data within the location of the distirct
## group the data by count number appeareance 
join_df_orig = telangana_fires.sjoin(Telangana)
join_df = join_df_orig.rename(columns={"NAME_3": "adm_name"})
join_df = join_df[["adm_name","year"]]
join_df = join_df.groupby(["adm_name","year"],as_index=False).value_counts()
join_df_orig[join_df_orig["NAME_3"] == "Achampet"]
join_df.head(10)

,adm_name,year,count
0,Achampet,2015,138
1,Achampet,2016,84
2,Achampet,2017,32
3,Achampet,2018,150
4,Achampet,2019,127
5,Achampet,2020,78
6,Achampet,2021,243
7,Adilabad,2015,2
8,Adilabad,2016,10
9,Adilabad,2017,17


In [401]:
## output data to json file
join_df.to_json("output1.json")